In [1]:
#import libraries

import csv
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
driver = webdriver.Chrome()

In [3]:
url = 'https://amazon.com'
driver.get(url)

In [4]:
#Define function to get the url using an input search term

def get_url(search_term):
    
    template = 'https://www.amazon.com/s?k={}&sprefix=migraine%2Caps%2C140&ref=nb_sb_ss_ts-doa-p_1_8'
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [5]:
url = get_url('migraine relief cap')
driver.get(url)

In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
results = soup.find_all('div', {'data-component-type': 's-search-result'})
len(results)

60

In [7]:
item = results[0]
atag = item.h2.a
description = atag.text.strip()
url = 'https://amazon.com' + atag.get('href')

In [8]:
price_parent = item.find('span', 'a-price')
price = price_parent.find('span', 'a-offscreen').text

In [9]:
rating = item.i.text

In [10]:
review_count = item.find('span', {'class':'a-size-base s-underline-text'}).text
print(review_count)

18,756


In [11]:
#Function to apply code above to every item on page
#Some listings may not have all items to return
#If no price skip record

def extract_record(item):
    
    #Description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://amazon.com' + atag.get('href')
    
    #Price
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    #Review Count
    try:
        rating = item.i.text
        review_count = item.find('span', {'class':'a-size-base s-underline-text'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

In [13]:
#Records list to contain all records on page
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [17]:
#Print first record on page
records[0]

('TheraICE Rx Form Fitting Gel Ice Headache / Migraine Relief Hat, Cold Therapy Migraine Relief Cap / Migraine Ice Head Wrap Ice Pack Mask, Cold Compress Headache Relief Cap for Tension, Sinus & Stress',
 '$36.95',
 '4.6 out of 5 stars',
 '18,756',
 'https://amazon.com/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A0431364D7AJD80MN94N&qualifier=1670324802&id=170761706596616&widgetName=sp_atf&url=%2FTheraICE-Rx-Headache-Comfortable-Stretchable%2Fdp%2FB082WN9NJL%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dmigraine%2Brelief%2Bcap%26qid%3D1670324802%26sprefix%3Dmigraine%252Caps%252C140%26sr%3D8-1-spons%26psc%3D1')

In [16]:
#Print all prices on page

for row in records:
    print(row[1])

$36.95
$27.99
$24.99
$16.99
$19.84
$24.99
$24.99
$27.99
$36.95
$26.99
$16.99
$27.99
$23.99
$17.59
$9.89
$11.99
$12.99
$27.99
$26.99
$13.99
$25.99
$11.99
$19.99
$24.99
$21.99
$26.99
$39.99
$15.99
$13.99
$22.99
$26.99
$19.99
$16.99
$17.99
$17.59
$15.99
$19.99
$27.99
$17.89
$17.99
$12.99
$11.99
$41.99
$13.99
$17.21
$16.99
$26.99
$16.96
$19.99
$19.99
$38.99
$25.99
$19.99
$12.99
$27.99
$10.99
$19.99
$21.99
$26.99
$32.99


In [18]:
#Amazon adds page# to url when searching through. Adjust function to apply this principal

def get_url(search_term):
    
    template = 'https://www.amazon.com/s?k={}&sprefix=migraine%2Caps%2C140&ref=nb_sb_ss_ts-doa-p_1_8'
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    #Placeholder
    url += '&page{}'
    
    return url

In [19]:
# Define function to do all
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_term):
    
    template = 'https://www.amazon.com/s?k={}&sprefix=migraine%2Caps%2C140&ref=nb_sb_ss_ts-doa-p_1_8'
    search_term = search_term.replace(' ', '+')
    
    url = template.format(search_term)
    
    #Placeholder
    url += '&page{}'
    
    return url

def extract_record(item):
    
    #Description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://amazon.com' + atag.get('href')
    
    #Price
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    
    #Review Count
    try:
        rating = item.i.text
        review_count = item.find('span', {'class':'a-size-base s-underline-text'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    
    result = (description, price, rating, review_count, url)
    
    return result

def main(search_term):
    driver = webdriver.Chrome()
    
    records = []
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    #save data to csv file
    
    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'ReviewCount', 'Url'])
        writer.writerows(records)

In [20]:
main('migraine relief cap')